# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#|export
from ghapi.all import GhApi
import requests, os
from fastcore.all import call_parse

In [ ]:
#|export
def get_id(owner,repo,pull_number):
    "Get the node id of the PR"
    api = GhApi(owner, repo)
    pr = api.pulls.get(pull_number)
    return pr.node_id

You can get the node id (which is used in graphql like this):

In [ ]:
#|eval: false
get_id('hamelsmu', 'lewinb', 3)

'PR_kwDOJKp2GM5MReJZ'

In [ ]:
assert 1 == 1

In [ ]:
#|export
def _mk_query(node_id):
    query = """mutation MyMutation {{
        enablePullRequestAutoMerge(input: {{pullRequestId: "{}", mergeMethod: MERGE}}) {{
            clientMutationId
             }}
    }}
    """.format(pr.node_id)
    return {'query': query}

In [ ]:
#|export
def _get_query(owner,repo,pull_number):
    return _mk_query(get_id(owner,repo,pull_number))

In [ ]:
#|export
@call_parse
def merge_pr(owner,repo,pull_number):
    token = os.getenv('GITHUB_TOKEN')
    assert token, 'You must set the GITHUB_TOKEN'
    query = _get_query(owner,repo,pull_number)
    headers = {'Authorization': f'Bearer {token}'}
    return requests.post(url="https://api.github.com/graphql",
              json={'query': query}, headers=headers)